<a href="https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Building%20with%20LangChain/Building_a_Conversational_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Minimal Conversational Agent</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain langchain-openai langgraph python-dotenv

In [7]:
import os
import langgraph
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


def check_inventory(product_name: str) -> str:
    product_name = product_name.lower()
    if "laptop" in product_name:
        return "The Pro-X Laptop has 15 units in stock."
    elif "monitor" in product_name:
        return "The Ultra HD Monitor is currently out of stock."
    else:
        return f"Could not find inventory details for {product_name}."

check_inventory_tool = Tool(
    name="check_inventory",
    func=check_inventory,
    description = "Returns the current stock count for a given product name."
)

summarization_middleware = SummarizationMiddleware(
    model="gpt-3.5-turbo",
    trigger=('tokens', 1000),
    keep=('messages', 5),
)

conversational_agent = create_agent(
    model=llm,
    tools=[check_inventory_tool],
    system_prompt="You are a helpful and detailed customer service agent. You must use your tools whenever necessary to answer questions about inventory.",
    middleware=[summarization_middleware],
    checkpointer=InMemorySaver(),
)


def run_chat(user_input: str):
    config = {"configurable": {"thread_id": "1001"}}
    response = conversational_agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config,
    )

    ai_response = response["messages"][-1].content
    return ai_response


while True:
    user_question = input("Enter your message: ")
    if user_question.lower() == "exit":
        break
    response = run_chat(user_question)
    print(response)

It seems that I couldn't find any inventory details for sugar. It's possible that it may not be listed under that name. Would you like me to check for any related products, such as "granulated sugar" or "brown sugar"?
Enter your message: exit
